# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [96]:
# imports

import requests
import json
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [97]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
model = "llama3.2"

In [98]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [99]:
payload = {
        "model": model,
        "messages": messages,
        "stream": False
    }

In [100]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [101]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [102]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [103]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [104]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [105]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [106]:

def summarize(url,model):
    website = Website(url)
    payload = {
        "model": model,
        "messages": messages_for(website),
        "stream": False
              }
    response = requests.post(url=OLLAMA_API, data=json.dumps(payload), headers=headers) # Convert payload to JSON string
    return response.json()

In [107]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url,model):
    summary = summarize(url,model)
    display(Markdown(summary['message']['content']))

In [109]:
display_summary("https://edwarddonner.com",model)

# Website Summary
## About the Author
The website is run by Edward Donner, a co-founder and CTO of Nebula.io, an AI startup that applies AI to help people discover their potential. He also has experience as the founder and CEO of another AI startup, untapt.

## Current News and Announcements
* **Upcoming Course: Connecting my courses – become an LLM expert and leader** (May 28, 2025)
* **AI Executive Briefing for 2025** (April 21, 2025)
* **The Complete Agentic AI Engineering Course** (January 23, 2025)
* **LLM Workshop: Hands-on with Agents – resources**

## Miscellaneous Information
Edward Donner is a hobbyist DJ and amateur electronic music producer. He enjoys reading Hacker News and nodding his head sagely to things he only half understands.

## Contact Information
To get in touch with Edward Donner, send an email to `ed [at] edwarddonner [dot] com`. You can also follow him on LinkedIn, Twitter, Facebook, or subscribe to his newsletter.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [66]:
!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 2.0 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 4.9 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏ 8.7 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏  14 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏  15 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏  16 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  26 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   3% ▕                  ▏  31 MB/1.1 GB                  pulling manifest 
pulling aabd4deb

In [117]:
model = "deepseek-r1:1.5b"
#model = "llama3.2"
display_summary("https://www.eenadu.net",model)

<think>
Alright, so I'm trying to understand this content about EENADU. The user provided a lot of information here, but I want to make sure I get all the key points.

First off, EENADU is an online news portal in Telugu, right? They have something called "Eenadu Epaper" and there's more on their site. It looks like they cover a wide range of topics including sports, business, crime, health, etc. Their homepage has sections that look like feature pages with various types of content.

I'm particularly interested in the news section. They mention something about "Sunday Magazine," which is highlighted as a section for kids and young adults. That's interesting because it suggests they cater to younger demographics, maybe more engaging and informative.

Looking at their website structure, I see categories like Business News, Crime News, Photo Gallery, etc. Each category seems to have multiple articles or photos spread out in sections. This makes the content organized but also quite spread out, which might make navigation a bit tricky for some users.

I notice there are also links to classifieds and news alerts. That's good because it tells people about various news events—whether they're happening soon or have certain information available. I wonder how detailed these sections are.

They have privacy policy and terms of service mentioned, which is a good idea to know how to use the site without breaking any rules. They also provide contact information for reporting issues, which can be helpful if someone feels there's content that's off.

I'm curious about their growth strategy. Since they're based in India, maybe they have digital advertising or some form of content curation. But I don't know much about the specifics unless I check more details.

The homepage also has a news feed feature with "Eenadu News." I wonder how frequent that is and if it's automated to save time for readers. Also, how many different pages are listed on their homepage? That could affect how quickly someone finds what they're looking for.

I'm thinking about how this content can be used beyond just reading. The photos in categories like Sports News might help with visual summaries of news articles. They have a feature sections that list various topics, so maybe they can create some interactive tools or educational resources.

I should also consider the audience demographics. Since EENADU is primarily targeting Telugu-speaking areas in India, but I'm not sure about other regions. Their content might be more suitable for this demographic and could face competition from English news portals if they expand globally.

Another thing to think about is how their language skills are perceived here. Being a language-native portal, it's probably aimed at promoting bilingual education or media training. They might have features that help translate content or highlight bilingual events.

I'm also interested in the visual style of EENADU Epaper. The design looks clean and professional with sections that seem to be well-organized. I wonder if they use any color schemes or typography that stands out, which could influence how information is presented.

Lastly, looking at their contact details, I see email addresses for editorial feedback and a way to report issues. They might have a support team as well, but without more info on that, it's hard to tell.

Overall, EENADU seems like a comprehensive news portal with diverse content sections, clear structure, and strong privacy protections. It's probably useful for both local communities in India and those looking for international news, especially from other parts of the world.
</think>

EENADU is an online news portal in Telugu that serves a wide audience, particularly targeting younger demographics through its "Eenadu Magazine" section. Here are the key aspects and considerations about EENADU:

1. **Content Coverage**: 
   - EENADU Epaper covers various topics including sports, business, crime, health, etc., with sections like Business News, Crime News, Photo Gallery, and more.
   - The homepage features "Sunday Magazine," especially for children and young adults.

2. **Website Structure**:
   - The website is divided into categories such as Business News, Crime News, Photo Gallery, etc., each containing multiple articles or photos spread out in sections. This organization aids in easy navigation but may present content in a somewhat spread-out manner.

3. **News Format**:
   - EENADU News has a "Eenadu News" section highlighted as a kid-friendly resource, indicating engagement and informativeness for younger audiences.

4. **Additional Sections**:
   - The site includes " Classifieds," news alerts, classified information, and news alerts in various languages like English. It also features a news feed with automatic updates, though specifics on frequency are not detailed.

5. **Privacy and Terms**:
   - EENADU adheres to the DNP Code of Ethics, emphasizing privacy, transparency, and proper usage. Contact information is provided for reporting issues.

6. **Growth and Strategy**:
   - Although specific strategies like digital advertising aren't detailed, they suggest a focus on local languages, possibly through bilingual education content or visual elements in photos.

7. **Audience Demographics**:
   - Targetting Telugu-speaking areas in India, EENADU's content is likely aimed at promoting media training and bilingual education, with potential competition from English news portals.

8. **Visual Style**:
   - The website features a clean design with organized sections using professional fonts and colors, enhancing readability and user experience.

9. **Future Potential**:
   - Considering the diversity of topics covered, EENADU's content can be useful for local communities in India and might expand into international news, though specifics are unclear.

In summary, EENADU offers a diverse range of news content with strong privacy protections, suitable for both local and international audiences, particularly targeting younger demographics. Its visual style and organization make it accessible, while its focus on local languages facilitates bilingual education and media training.

In [ ]:
## deep seek model providied more detailed summary of the website where it tried to cover all the pages. 
##where as llama model doesn't. Also the deepseep model provided how it is thinking and reasoning to findout the answer.

So Far we have done via http call instead of that we can call via open AI

In [126]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
def call_from_openAI(model,url):
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    model = model
    website = Website(url)
    messages = messages_for(website)
    response = ollama_via_openai.chat.completions.create( model=model,messages=messages)
    print(response.choices[0].message.content)

In [129]:
#model = "deepseek-r1:1.5b"
model = "llama3.2"
url = "https://www.cnn.com"
call_from_openAI(model,url)

Here are the top news stories from CNN:

**Breaking News**

* Ukraine: Russia claims to have pushed into central Ukrainian region for first time
* Gaza: Colombia president vows to hunt 'mastermind' behind shooting of political rival, after 15-year-old arrested
* Business: Stocks rise as investors wait for economic data

**Politics**

* Trump: Encouraged Vance to respond diplomatically when asked about Musk
* Elections: 2025 Elections: CNN Polls

**Sports**

* Tennis: Alcaraz defeats Sinner to win back-to-back Roland Garros titles
* NBA: Trendlines: The smallest markets are winning the NBA finals

**Health**

* COVID-19: Experts say 'detox' after Covid vaccination is nonsense

**Business**

* Markets: Pre-markets, After-Hours, Fear & Greed

**Technology**

* Tech: How safe is online gaming for kids? Roblox's CEO has some thoughts
* Innovation: How AI analysis can give new age estimates for Dead Sea Scrolls

**Travel**

* Travel: Why hikers are drawn to these stunning canyons despite the